In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum, sign
from IPython.display import display, HTML, Math

In [2]:
delta_b_negative = False
discrete = False
simp = True

In [3]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

## Equalities

In [4]:
# affine
def deltaBySpot_(asset, s,v, b, w, j, e, Delta, a, min):
  f = (s - (v + b) * w) / w
  if delta_b_negative and asset == 'b':
    f = -f
  if discrete:
    f = ceiling(f)
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
deltaBySpot['b']

-b_b + s_b/w_b - v_b

In [5]:
# convex
def spotByExp_(s, v, b, w, j, e, Delta, a, min):
  f = a * ((1 + 1/j) ** e)
  if discrete:
    f = floor(f)
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*((j_b + 1)/j_b)**e_b

In [6]:
# convex if not delta_b_negative
def deltaByExp_(asset, s, v, b, w, j, e, Delta, a, min):
  f = deltaBySpot[asset].subs(s, spotByExp[asset])
  if simp:
    f = simplify(f)
  return f

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) - b_b - v_b

In [7]:
deltaByExp['s']

a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - v_s

## Target function

we want to minimize the spot price, given a set of exponents for the buying and selling asset each. (Effective price wasn't convex)

In [8]:
spot = s_s / s_b
spot

s_s/s_b

In [9]:
H = hessian(spot, [s_s, s_b])
H

Matrix([
[        0,    -1/s_b**2],
[-1/s_b**2, 2*s_s/s_b**3]])

In [10]:
H.det() # concave at best

-1/s_b**4

In [11]:
spotByExps = spot.subs(s_s, spotByExp['s']).subs(s_b, spotByExp['b'])
if simp:
  spotByExps = simplify(spotByExps)
spotByExps

a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s/a_b

In [12]:
H = hessian(spotByExps, [e_b, e_s])
H

Matrix([
[                a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))**2/a_b, a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))*log((j_s + 1)/j_s)/a_b],
[a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))*log((j_s + 1)/j_s)/a_b,                 a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log((j_s + 1)/j_s)**2/a_b]])

In [13]:
H[0,0].simplify()

a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))**2/a_b

In [14]:
H.det().simplify()

0

- top-left of the hessian is positive
- determinant is zero

=> convex, but not strictly so

## Inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds given by our equation
- need to buy and sell minimum amounts
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not exceed expLimit (TODO)

### value cannot decrease

In [15]:
a0Buying = s_s * deltaBySpot['b'] # < 0 if delta_b_negative, > 0 otherwise
if delta_b_negative:
    a0Buying = -a0Buying # > 0
if simp:
  a0Buying = simplify(a0Buying)
a0Buying

s_s*(s_b - w_b*(b_b + v_b))/w_b

In [16]:
a0Selling = s_b * deltaBySpot['s'] # > 0
if simp:
  a0Selling = simplify(a0Selling)
a0Selling

s_b*(s_s - w_s*(b_s + v_s))/w_s

In [17]:
a0Bound = a0Selling - a0Buying # >=! 0 (because it's concave, as is it's negation)
if simp:
  a0Bound = simplify(a0Bound)
a0Bound

(s_b*w_b*(s_s - w_s*(b_s + v_s)) - s_s*w_s*(s_b - w_b*(b_b + v_b)))/(w_b*w_s)

In [18]:
H = hessian(a0Bound, [s_s, s_b])
H

Matrix([
[                    0, (w_b - w_s)/(w_b*w_s)],
[(w_b - w_s)/(w_b*w_s),                     0]])

In [19]:
det = H.det()
det = simplify(det)
det

-1/w_s**2 + 2/(w_b*w_s) - 1/w_b**2

In [20]:
det_ = -((1/w_b) - (1/w_s))**2 # nonpositive
det_

-(-1/w_s + 1/w_b)**2

In [21]:
simplify(det_ - det)

0

- topleft zero
- determinant nonpositive

=> non-strictly concave => suitable as inequality-constraint, but inverted (g >= 0)

- not affine, because then the entire Hessian would be filled with zeroes

### exponents cannot result in spot prices that are better than the amm-prices

In [22]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))

In [23]:
expBound['s']

-e_s + log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))

- both clearly affine wrt e

=> suitable as all kinds of constraints

### spot prices cannot exceed maxInteger

In [106]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

maxSpotBound_b = spotByExp['b'] - I_max # <=! 0
if simp:
  maxSpotBound_b = simplify(maxSpotBound_b)
maxSpotBound_b

-I_max + a_b*(j_b + 1)**e_b/j_b**e_b

In [107]:
maxSpotBound_s = spotByExp['s'] - I_max # >=! 0
if simp:
  maxSpotBound_s = simplify(maxSpotBound_s)
maxSpotBound_s

-I_max + a_s*(j_s + 1)**e_s/j_s**e_s

In [108]:
maxSpotBound_b.diff(e_b).simplify().diff(e_b).simplify()

-log((j_b/(j_b + 1))**log(((j_b + 1)/j_b)**(a_b*(j_b + 1)**e_b/j_b**e_b)))

In [109]:
maxSpotBound_s.diff(e_s).simplify().diff(e_s).simplify()

-log((j_s/(j_s + 1))**log(((j_s + 1)/j_s)**(a_s*(j_s + 1)**e_s/j_s**e_s)))

- both are positive

=> convex => suitable as regular inequality-constraints

(the reverse formulation also works, meaning negate it => becomes convave => suitable as inverted inequality-constraints)

### need to swap at least minimum amounts

In [110]:
if delta_b_negative:
  raise Exception('not checked for convexity/concavity')
else:
  minAmntBound_b = min_b - deltaByExp['b'] # <=! 0
if simp:
  minAmntBound_b = simplify(minAmntBound_b)
minAmntBound_b

-a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) + b_b + min_b + v_b

In [111]:
minAmntBound_s = min_s - deltaByExp['s'] # <=! 0
if simp:
  minAmntBound_s = simplify(minAmntBound_s)
minAmntBound_s

-a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) + b_s + min_s + v_s

In [112]:
minAmntBound_b.diff(e_b).simplify().diff(e_b).simplify()

-a_b*((j_b + 1)/j_b)**e_b*log(j_b/(j_b + 1))**2/w_b

In [113]:
minAmntBound_s.diff(e_s).simplify().diff(e_s).simplify()

-a_s*((j_s + 1)/j_s)**e_s*log(j_s/(j_s + 1))**2/w_s

- both are negative

=> concave => not suitable as regular inequality-constraints!

In [114]:
if delta_b_negative:
  raise Exception('not checked for convexity/concavity')
else:
  minAmntBound_b = deltaByExp['b'] - min_b # >=! 0
if simp:
  minAmntBound_b = simplify(minAmntBound_b)
minAmntBound_b

a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) - b_b - min_b - v_b

In [115]:
minAmntBound_s = deltaByExp['s'] - min_s # >=! 0
if simp:
  minAmntBound_s = simplify(minAmntBound_s)
minAmntBound_s

a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - min_s - v_s

In [116]:
minAmntBound_b.diff(e_b).simplify().diff(e_b).simplify()

-log((j_b/(j_b + 1))**log(((j_b + 1)/j_b)**(a_b*(j_b + 1)**e_b/(j_b**e_b*w_b))))

In [117]:
minAmntBound_s.diff(e_s).simplify().diff(e_s).simplify()

-log((j_s/(j_s + 1))**log(((j_s + 1)/j_s)**(a_s*(j_s + 1)**e_s/(j_s**e_s*w_s))))

- both are positive

=> convex => not suitable as inverted inequality-constraints!

In [118]:
print(deltaByExp['s'])

a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - v_s


In [119]:
e_s



e_s

!>=

In [120]:
tmp = log((min_s + b_s + v_s) * w_s / a_s) / log((j_s + 1)/j_s)
tmp

log(w_s*(b_s + min_s + v_s)/a_s)/log((j_s + 1)/j_s)

TODO verify this reformulation

In [121]:
simplify(tmp)

log((w_s*(b_s + min_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))

In [122]:
simplify(expBound['s'] + e_s) # lower bound for e_s

log((w_s*(b_s + v_s)/a_s)**(1/(-log(j_s) + log(j_s + 1))))

- the above (derived from min_s requirement) is strictly larger, therefore more binding => obsoletes expBound['s']

- more importantly, e_s - the above term is affine, thus suitable for all kinds of constraints

In [126]:
minAmntBound_s = tmp - e_s # <=! 0
if simp:
  minAmntBound_s = simplify(minAmntBound_s)
minAmntBound_s

log((w_b*(b_b + min_b + v_b)/(a_b*((j_b + 1)/j_b)**e_s))**(1/log((j_b + 1)/j_b)))

In [127]:
minAmntBound_s.diff(e_s).simplify().diff(e_s).simplify()

0

In [128]:
if delta_b_negative:
  raise Exception('not checked for convexity/concavity')

tmp = log((min_b + b_b + v_b) * w_b / a_b) / log((j_b + 1)/j_b)
tmp

log(w_b*(b_b + min_b + v_b)/a_b)/log((j_b + 1)/j_b)

TODO verify this reformulation as well (created by s/_s/_b)

In [129]:
minAmntBound_b = tmp - e_b # <=! 0
if simp:
  minAmntBound_b = simplify(minAmntBound_b)
minAmntBound_b

log((w_b*(b_b + min_b + v_b)/(a_b*((j_b + 1)/j_b)**e_b))**(1/log((j_b + 1)/j_b)))

### need to swap at most maximum amounts

In [130]:
if delta_b_negative:
  raise Exception('not checked for convexity/concavity')
else:
  maxAmntBound_b = deltaByExp['b'] - b_b # <=! 0
if simp:
  maxAmntBound_b = simplify(maxAmntBound_b)
maxAmntBound_b

a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) - 2*b_b - v_b

In [131]:
maxAmntBound_s = deltaByExp['s'] - max_s # <=! 0
if simp:
  maxAmntBound_s = simplify(maxAmntBound_s)
maxAmntBound_s

a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - max_s - v_s

In [132]:
maxAmntBound_b.diff(e_b).simplify().diff(e_b).simplify()

-log((j_b/(j_b + 1))**log(((j_b + 1)/j_b)**(a_b*(j_b + 1)**e_b/(j_b**e_b*w_b))))

In [133]:
maxAmntBound_s.diff(e_s).simplify().diff(e_s).simplify()

-log((j_s/(j_s + 1))**log(((j_s + 1)/j_s)**(a_s*(j_s + 1)**e_s/(j_s**e_s*w_s))))

- both are positive

=> convex => suitable as regular inequality-constraints

(the reverse formulation would probably also work)

### total number of multiplications of both exponentiations cannot exceed expLimit

In [134]:
# expLimit, aka maximum total number of multiplications for both exponents
m_max = symbols('m_max', positive=True, integer=True)

def numMultsLower_(asset, s,v, b, w, j, e, Delta, a, min):
  return floor(log(e, 2))

# the best number of multiplications we can get by increasing e
def bestMultsAhead_(asset, s,v, b, w, j, e, Delta, a, min):
  return log(e, 2) # NOTE/TODO just dropping the ceiling here nonchalantly
# return ceiling(log(e, 2))

bestMultsAhead = {asset: bestMultsAhead_(asset, *all_symbols[asset]) for asset in assets}
bestMultsAhead['b']

log(e_b)/log(2)

In [135]:
# optimistic estimate. This relies on the vague intuition that exponents will be minimal from the other constraints, so best we can do is increase them if this one is violated
optimisticMultsBound = bestMultsAhead['b'] + bestMultsAhead['s'] # <=! 0
if simp:
  optimisticMultsBound = simplify(optimisticMultsBound)
optimisticMultsBound

(log(e_b) + log(e_s))/log(2)

In [136]:
H = hessian(optimisticMultsBound, [e_b, e_s])
H

Matrix([
[-1/(e_b**2*log(2)),                  0],
[                 0, -1/(e_s**2*log(2))]])

In [137]:
H.det()

1/(e_b**2*e_s**2*log(2)**2)

- determinant and topleft clearly different signs

=> neither convex nor concave => unsuitable as constraints! => TODO

## Lagrangian

In [139]:
# Decision variables
xs = [e_b, e_s]

# Equality-Constraints
hs = [] 

# Inequality-Constraints
gs = [
    expBound['b'],
    # expBound['s'], # obsoleted by minAmntBound_s, see above
    maxSpotBound_b,
    maxSpotBound_s,
    minAmntBound_b,
    minAmntBound_s,
    maxAmntBound_b,
    maxAmntBound_s,
    # optimisticMultsBound, # neither concave nor convex yet
] 

# inverted Inequality-Constraints
ginvs = [
    a0Bound,
    # optimisticMultsBound, # neither concave nor convex yet
] 

# Lagrange multipliers for equality- and inequality-constraints
mus = symbols('mu_1:{}'.format(len(hs)+1))
lambdas = symbols('lambda_1:{}'.format(len(gs)+1), nonnegative=True)
Lambdas = symbols('Lambda_1:{}'.format(len(ginvs)+1), nonpositive=True)

# Objective function
f = spotByExps

# Lagrangian
L = f + sum([mu * h for mu, h in zip(mus, hs)]) + sum([lambda_ * g for lambda_, g in zip(lambdas, gs)]) + sum([Lambda_ * ginv for Lambda_, ginv in zip(Lambdas, ginvs)])
if simp:
    L = simplify(L)
L

Lambda_1*(s_b*w_b*(s_s - w_s*(b_s + v_s)) - s_s*w_s*(s_b - w_b*(b_b + v_b)))/(w_b*w_s) + lambda_1*(e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))) - lambda_2*(I_max - a_b*(j_b + 1)**e_b/j_b**e_b) - lambda_3*(I_max - a_s*(j_s + 1)**e_s/j_s**e_s) + lambda_4*log((w_b*(b_b + min_b + v_b)/(a_b*((j_b + 1)/j_b)**e_b))**(1/log((j_b + 1)/j_b))) + lambda_5*log((w_b*(b_b + min_b + v_b)/(a_b*((j_b + 1)/j_b)**e_s))**(1/log((j_b + 1)/j_b))) - lambda_6*(-a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) + 2*b_b + v_b) - lambda_7*(-a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) + b_s + max_s + v_s) + a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s/a_b

In [140]:
# Equations from stationarity

def custom_diff(f, n):
    diff = f.diff(n)
    # diff = floor(f.subs(n, n+1) - f)
    if simp:
        diff = simplify(diff)
    return diff

equations = []
for x in xs:
  dL_dx = custom_diff(L, x)
  equation = Eq(dL_dx, 0)
  if simp: # results in error from the bestMultsAhead-constraint
    equation = simplify(equation)
  display(equation)
  equations.append(equation)

Eq((1/(j_b**2*(j_b + 1)))**e_b*(-a_b**2*j_s**e_s*lambda_2*w_b*(j_b*(j_b + 1)**2)**e_b*(log(j_b) - log(j_b + 1)) - a_b**2*j_s**e_s*lambda_6*(j_b*(j_b + 1)**2)**e_b*(log(j_b) - log(j_b + 1)) + a_b*j_s**e_s*w_b*(j_b**2*(j_b + 1))**e_b*(lambda_1 - lambda_4) + a_s*j_b**(3*e_b)*w_b*(j_s + 1)**e_s*(log(j_b) - log(j_b + 1)))/(a_b*j_s**e_s*w_b), 0)

Eq((-a_b*a_s*lambda_3*w_s*(j_s**2*(j_s + 1))**e_s*(j_b + 1)**e_b*(log(j_s) - log(j_s + 1)) - a_b*a_s*lambda_7*(j_s**2*(j_s + 1))**e_s*(j_b + 1)**e_b*(log(j_s) - log(j_s + 1)) - a_b*j_s**(3*e_s)*lambda_5*w_s*(j_b + 1)**e_b - a_s*j_b**e_b*w_s*(j_s**2*(j_s + 1))**e_s*(log(j_s) - log(j_s + 1)))/(a_b*j_s**(3*e_s)*w_s*(j_b + 1)**e_b), 0)

In [141]:
# Primal feasibility conditions for equality-constraints
for h in hs:
    equation = Eq(h, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    equations.append(equation)

In [142]:
# Complementary slackness conditions for regular inequality-constraints
for lambda_, g in zip(lambdas, gs):
    equation = Eq(lambda_ * g, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    equations.append(equation)

Eq(lambda_1*(e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))), 0)

Eq(I_max*lambda_2 - a_b*lambda_2*(j_b + 1)**e_b/j_b**e_b, 0)

Eq(I_max*lambda_3 - a_s*lambda_3*(j_s + 1)**e_s/j_s**e_s, 0)

Eq(log(((j_b**e_b*w_b*(b_b + min_b + v_b))**(1/log((j_b + 1)/j_b))/(a_b*(j_b + 1)**e_b)**(1/log((j_b + 1)/j_b)))**lambda_4), 0)

Eq(log(((j_b**e_s*w_b*(b_b + min_b + v_b))**(1/log((j_b + 1)/j_b))/(a_b*(j_b + 1)**e_s)**(1/log((j_b + 1)/j_b)))**lambda_5), 0)

Eq(-a_b*lambda_6*(j_b + 1)**e_b/(j_b**e_b*w_b) + 2*b_b*lambda_6 + lambda_6*v_b, 0)

Eq(-a_s*lambda_7*(j_s + 1)**e_s/(j_s**e_s*w_s) + b_s*lambda_7 + lambda_7*max_s + lambda_7*v_s, 0)

In [143]:
# Complementary slackness conditions for inverted inequality-constraints
for Lambda_, ginv in zip(Lambdas, ginvs):
    equation = Eq(Lambda_ * ginv, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    equations.append(equation)

Eq(Lambda_1*(s_b*w_b*(s_s - w_s*(b_s + v_s)) - s_s*w_s*(s_b - w_b*(b_b + v_b)))/(w_b*w_s), 0)

In [144]:
def verify_inequality_constraints(solution, vars, gs):
    subs_dict = dict(zip(vars, solution))
    for g in gs:
        if Gt(g.subs(subs_dict), 0) == True:
            return False
    return True

In [145]:
def verify_inverted_inequality_constraints(solution, vars, ginvs):
    subs_dict = dict(zip(vars, solution))
    for ginv in ginvs:
        if Lt(ginv.subs(subs_dict), 0) == True:
            return False
    return True

In [146]:
# Solve
vars = xs + list(mus) + list(lambdas) + list(Lambdas)
solutions = nonlinsolve(equations, vars)
len(solutions)

6

In [147]:
solutions_ = [solution for solution in solutions if verify_inequality_constraints(solution, vars, gs)]
len(solutions_)

6

In [148]:
solutions__ = [solution for solution in solutions_ if verify_inverted_inequality_constraints(solution, vars, ginvs)]
len(solutions__)

6

In [152]:
for i, solution in enumerate(solutions__):
    print("solution", i+1)
    for var, expression in zip(vars, solution):
        display(Eq(var, expression))

solution 1


True

True

Eq(lambda_1, 0)

Eq(lambda_2, 0)

...

True

True

Eq(lambda_6, 0)

Eq(lambda_7, 0)

Eq(Lambda_1, 0)